In [28]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.applications import VGG16
from keras.optimizers import Adam

# Завантаження датасету Fashion MNIST

In [29]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Розмір зображень, що будуть використовуватись в моделі

In [30]:
image_size = (32, 32)

# Зміна розмірності зображень та повторення каналів

In [31]:
train_images = tf.image.resize(train_images[..., np.newaxis], image_size)
train_images = np.repeat(train_images, 3, axis=-1)
test_images = tf.image.resize(test_images[..., np.newaxis], image_size)
test_images = np.repeat(test_images, 3, axis=-1)

# Масштабування пікселів до діапазону [0, 1]

In [32]:
train_images = np.array(train_images) / 255.0
test_images = np.array(test_images) / 255.0

# Кількість класів

In [33]:
num_classes = 10

# Перетворення міток в бінарний формат

In [34]:
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Завантаження базової моделі VGG16 без верхніх повнозв'язних шарів та ваг

In [35]:
vgg_base = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(image_size[0], image_size[1], 3)
    )

# Заморожування базової моделі VGG16, щоб не навчати її ваги

In [36]:
vgg_base.trainable = False

# Будуємо модель

In [37]:
model = Sequential()

model.add(vgg_base)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

# Компіляція моделі

In [38]:
optimizer = Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
              )

# Навчання моделі

In [39]:
batch_size = 64
epochs = 20

history = model.fit(
    train_images,
    train_labels,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(test_images, test_labels)
    )

Epoch 1/20
938/938 [==============================] - 882s 939ms/step - loss: 0.5513 - accuracy: 0.8032 - val_loss: 0.4488 - val_accuracy: 0.8364
Epoch 2/20
938/938 [==============================] - 881s 939ms/step - loss: 0.4182 - accuracy: 0.8456 - val_loss: 0.4031 - val_accuracy: 0.8522
Epoch 3/20
938/938 [==============================] - 877s 935ms/step - loss: 0.3859 - accuracy: 0.8568 - val_loss: 0.3888 - val_accuracy: 0.8581
Epoch 4/20
938/938 [==============================] - 872s 929ms/step - loss: 0.3637 - accuracy: 0.8651 - val_loss: 0.3832 - val_accuracy: 0.8586
Epoch 5/20
938/938 [==============================] - 849s 906ms/step - loss: 0.3465 - accuracy: 0.8717 - val_loss: 0.3680 - val_accuracy: 0.8681
Epoch 6/20
938/938 [==============================] - 872s 930ms/step - loss: 0.3346 - accuracy: 0.8750 - val_loss: 0.3627 - val_accuracy: 0.8684
Epoch 7/20
938/938 [==============================] - 872s 930ms/step - loss: 0.3238 - accuracy: 0.8789 - val_loss: 0.3675 -

# Оцінка точності моделі

In [40]:
_, accuracy = model.evaluate(test_images, test_labels)
print(f'Точність на тестових даних: {accuracy * 100:.2f}%')


313/313 [==============================] - 123s 393ms/step - loss: 0.3508 - accuracy: 0.8779
Точність на тестових даних: 87.79%


ВИСНОВКИ:
Отримана точність багатошарової мережі на тестових даних становить 92.41%.

Порівнюючи цю точність з точністю згорткової мережі, яку ми навчали у поточному завданні, яка становила 87.79% на тестових даних, ми бачимо, що багатошарова  мережа показала кращі результати.

Це може бути незвичайним, оскільки в загальному згорткові мережі, як правило, добре показуються на завданнях класифікації зображень.
Також слід зазначити, що тренування згорткової мережі зайняло значно більше часу не зважаючи на меншу кількість епох.